# Recomanador Simple

## 1. INFORMACIÓ 

### 1.1. Abans de començar...

**\+ Durant la pràctica, solament es podran fer servir les següents llibreries**:

`Pandas, Numpy, Itertools`

*Nota: A més de les que ja es troben presents en la 1a cel·la i funcions natives de Python*

**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**

Això no implica però que els hàgiu de fer servir. És a dir, que la funció tingui un paràmetre anomenat `df` no implica que l'hàgiu de fer servir, si no ho trobeu convenient.

**\+ En les funcions, s'especifica que serà i de quin tipus cada un dels paràmetres, cal respectar-ho**

Per exemple (ho posarà en el pydoc de la funció), `df` sempre serà indicatiu del `Pandas.DataFrame` de les dades. Durant la correcció, els paràmetres (i específicament `df`) no contindran les mateixes dades que en aquest notebook, si bé si seran del mateix tipus! Per tant, no us refieu de què tinguin, per exemple, el mateix nombre de files.

### 1.2. Consum general

La base de dades [movielens-1M](http://www.grouplens.org/node/73) conté 1,000,209 puntuacions de 3.900 pel·licules fetes l'any 2000 per 6.040 usuaris anònims del recomanador online [MovieLens](http://www.movielens.org/). 

El consum total de tots els usuaris s'hi pot trobar al document "ratings.dat" el format següent:

    UserID::MovieID::Rating::Timestamp

- **UserIDs** usuaris amb id's entre 1 i 6040 
- **MovieIDs** pelis amb id's entre 1 i 3952
- **Ratings** són les puntuacions en una escala de 5 estrelles.
- **Timestamp** representat en segons

> Cada usuari té com a mínim 20 interaccions consumides.

### 1.3. Usuaris



Al fitxer "users.dat" hi trobem la informació referent a cadascun dels usuaris en el següent format:

        UserID::Gender::Age::Occupation::Zip-code

- **Gender** ve donat per "M" per home i "F" per dona.
- **Age** està representada de la següent forma:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- **Occupation** es tria entre les següents opcions:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"

> Els usuaris han donat la informació voluntariament. Així doncs, alguns usuaris poden no tenir informació.


### 1.4. Películes



Al fitxer "movies.dat" hi trobem la informació referent a cadascuna de les películes en el següent format:

        MovieID::Title::Genres

- **Titles** són identics als titols de la base de dades IMDB, incloent l'any de llançament.
- **Genres** de les películes estan separats i seleccionats d'entre els següents:

	* Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western

> Algunes películes poden tenir el ID malament degut a duplicats accidentals.
>
> Les películes s'han entrat manualment, així que altres inconsistencies poden existir. 

## 2. COMENCEM: explorem les dades!!

### 2.1. Descarregar i llegir dades

+ Baixa't els fitxers que composen la base de dades i els còpies al teu directori de treball. 

+ Llegeix les tres taules de la base de dades en tres DataFrames de pandas amb aquest codi:

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip

In [ ]:
import pandas as pd
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python', encoding='latin-1')


### 2.2. Inspecció de les taules

In [ ]:
users[:10]

In [ ]:
users[-10:]

In [ ]:
ratings[-10:]

In [ ]:
ratings[:10]

In [ ]:
ratings.sort_values('movie_id')[:5]


In [ ]:
movies[:5]

In [ ]:
ratings[:5]

### **Exemple:** Fent càlculs sobre DataFrames.

Suposa que volem calcular les **puntuacions mitjanes d'una pel·licula per sexe o edat**. 


El primer pas a obtenir una única estructura que contingui tota la informació. Per fer-ho podem usar la funció ``merge`` de pandas. Aquesta funció infereix automàticament quines columnes ha d'usar per fer el ``merge`` basant-se en els noms que fan intersecció:

In [ ]:
data = pd.merge(pd.merge(ratings, users), movies)

# Visualitzem la taula ordenada per identificar d'usuari
data.sort_values(by='user_id')[:10]

La funció ``iloc`` ens permet obtenir un subconjunt de files i/o columnes:

In [ ]:
data.iloc[3:5]

Els índexs Boolans ens permeten seleccionar una part de la taula que compleix una condició.

In [ ]:
# comptem quin tant per cent de ratings estan fets per una dona
print(data[data['gender']=='F']['rating'].count()/float(data['rating'].count()), '%')

Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per edat** podem usar el mètode ``pivot_table`` que és una forma de "canviar" la forma de la taula especificant quin valor agregat (mitjançant una funció predefinida) hi volem en funció dels valors de dues columnes:

In [ ]:
mean_ratings = data.pivot_table(values= 'rating', index='title', columns='age', aggfunc='mean')
mean_ratings[:10]

Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per sexe**:

In [ ]:
mean_ratings = data.pivot_table('rating', index='title',columns='gender', aggfunc='mean')
mean_ratings[:10]

Si volgéssim fer càlculs només sobre les pel·licules que han rebut **al menys** 250 puntuacions, primer hem de construir una taula amb el nombre d'avaluacions de cada títol. Per fer-ho, agruparem les dades per títol (amb el mètode ``groupby``) i usarem ``size()`` el nombre.

El mètode ``groupby`` implenta un o més d'aquests processos:

+ Dividir les dades segons algun criteri.
+ Aplicar una funció a cada grup.
+ Combinar els resultats en una estructura de dades.

In [ ]:
ratings_by_title = data.groupby('title').size()
print(ratings_by_title)

Llavors podem crear un índex amb els títols amb més de 250 avaluacions.

In [ ]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]
active_titles

L'índex de títols que reben al menys 250 puntuacions es pot fer servir per seleccionar les files de ``mean_ratings``: 

In [ ]:
mean_ratings = mean_ratings.loc[active_titles]
mean_ratings

Per veure els films més valorats per les dones, podem ordenar per la columna F de forma descendent:

In [ ]:
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)
top_female_ratings[:10]

Suposem ara que volem les pel·licules que estan valorades de forma més diferent entre homes i dones. Una forma d'obtenir-ho és afegir una columna a ``mean_ratings`` que contingui la diferència en mitjana i llavors ordenar:

In [ ]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

Ordenant per ``diff`` ens dóna les pel·licules ben valorades per les dones que presenten més diferència entre homes i dones:

In [ ]:
sorted_by_diff = mean_ratings.sort_values(by='diff')
sorted_by_diff[:15]

Invertint l'ordre de les files i fent un ``slicing`` de les 15 files superiors obtenim les pel·licules ben valorades pels homes que no han agradat a les dones: 

In [ ]:
sorted_by_diff[::-1][:15]

Si volguéssim les pel·licules que han generat puntuacions més discordants, independentment del gènere, podem fer servir la variança o la desviació estàndard de les puntuacions: 

In [ ]:
# Standard deviation of rating grouped by title
rating_std_by_title = data.groupby('title')['rating'].std()
# Filter down to active_titles
rating_std_by_title = rating_std_by_title.loc[active_titles]
rating_std_by_title.sort_values(ascending=False)[:10]

### Important: Temes de rendiment

In [ ]:
%timeit data['title'] 
%timeit data.title 
%timeit data[['title']] 

In [ ]:
type(data[['title']])

In [ ]:
type(data.title)

## 3. EXERCICIS

### 3.1. PRIMER EXERCICI 

Donada la taula ``data``, calcula la puntuació mitjana de cada usuari. 

In [ ]:
!ls

In [ ]:
data_folder = 'ml-1m'

In [ ]:
import pandas as pd
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table(f'{data_folder}/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(f'{data_folder}/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table(f'{data_folder}/movies.dat', sep='::', header=None, names=mnames, engine='python',encoding='latin-1')

data = pd.merge(pd.merge(ratings, users), movies)

# la vostra solució aquí

+ Quina és la pel·lícula més ben puntuada (en mitja) pels usuaris? (Guarda aquest valor en una variable ``string``). 

In [ ]:
# la vostra solució aquí

### 3.2. SEGON EXERCICI

Defineix una funció anomenada ``top_movie`` que donat un usuari ens retorni quina és la pel·lícula millor puntuada.<br> 


In [ ]:
# la vostra solució aquí
def top_movie(dataFrame,usr):
    return 0

print(top_movie(data,1))

### 3.3. TERCER EXERCICI

Construeix una funció que donat un dataframe et retorni el valor que cada usuari li ha donat a una peli. Això ho farem creant un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors siguin el rating donat.

In [ ]:
def build_counts_table(df):
    """
    Retorna un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors
    el nombre de vegades que un usuari ha vist una peli d'un `movie_id`
    
    :param df: DataFrame original 
    :return: DataFrame descrit adalt
    """
    
    # la vostra solució aquí
    
    return 0

In [ ]:
df_counts = build_counts_table(data)
df_counts

També programarem una funció que donada la taula anterior i dos id's (usuari i peli), extregui el valor donat:

In [ ]:
def get_count(df, user_id, movie_id):
    """
    Retorna el nombre de vegades que un usuari ha comprat en un `movie_id`
    
    :param df: DataFrame retornat per `build_counts_table`
    :param user_id: ID de l'usuari
    :param movie_id: ID de la peli
    :return: Enter amb el nombre de vegades que ha comprat
    """
    
    # la vostra solució aquí
    
    return 0

get_count(df_counts, 0, 0)

### 3.4. QUART EXERCICI

In [ ]:
data.nunique()

Si observem el nombre total d'usuaris unics i de pelicules úniques, podem observar que els id's dels usuaris van de 1 a 6040. Normalment hauriem d'intentar que comencessin al nombre 0, anant de 0 a 6039. 

**Què passa amb els indexos de les pelis??**

> ANSWER

Usant la funció **pd.Categorical(*).codes**, re-indexa els id's dels usuaris i de les pelis perquè vagin de 0 a 6039 i de 0 a 3705 respectivament:

In [ ]:
# la vostra solució aquí

Per comprovar que tot sigui correcte i guardar correctament la taula **df_counts**, torna a calcular-la:

In [ ]:
df_counts = build_counts_table(data)
df_counts

### 3.5. CINQUÉ EXERCICI



Construeix una funció <b>distEuclid(x,y)</b> que implementi la distància Euclidiana entre dos vectors usant funcions de pandas. Escriu la funció que calculin la semblança entre dos usuaris segons aquesta estructura:

``def SimEuclid (DataFrame, User1, User2)``
    Calcular els **embeddings** de cada usuari, C1 i C2, amb les puntuacions dels ítems comuns que han puntuat el dos usuaris.
    Si no hi ha puntuacions en comú, retornar 0. Retornar ``1/(1+distEuclid(C1, C2))``

\
Avalueu amb la funció ``%timeit`` quant triguen aquests càlculs per un parell d'usuaris.   

> *Nota: Alguns d'aquests exercicis tenen temps de càlcul de l'ordre de minuts sobre tota la base de dades. Per desenvolupar els algorismes és recomanable treballar amb una versió reduïda de la base de dades.* 

> *Nota: **embedding** fa referencia a un vector de N dimensions que té com a funció representar (en aquest cas) els gustos dels usuaris. També es pot fer un embedding d'items i aleshores representarien el contingut d'aquest o caracteristiques semblants.*

---
#### Mesurament de similituds (petita teoria)

El primer pas per poder recomanar és definir una funció de similitud entre vectors. Siguin $x, y$ vectors, implementa la seva distància euclidiana:

* Distància euclidea (inversa): https://en.wikipedia.org/wiki/Euclidean_distance

$$sim(x, y) = \frac{1}{1 + \sqrt{\sum_i(x_i-y_i)^2}}\in [0, 1]$$

---

Per implementar qualsevol d'aquestes únicament es permet l'ús de:

* `np.sum`
* `np.sqrt`
* `np.power`
* `np.dot`
* `np.linalg.norm`
* `np.mean`

I s'ha de fer **sense bucles**.


In [ ]:
import math
import numpy as np
import pandas as pd

def distEuclid(x, y):
    """
    Retorna la distancia euclidiana de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la distancia euclidiana
    """
    
    # la vostra solució aquí
    
    return 0

def SimEuclid(DataFrame, User1, User2):
    """
    Retorna un score que representa la similitud entre user1 i user2 basada en la distancia euclidiana
    
    :param DataFrame: dataframe que conté totes les dades
    :param User1: id user1
    :param User2: id user2
    :return : Escalar (float) corresponent al score
    """
    
    # la vostra solució aquí
    
    return 0

In [ ]:
# Execute functions
print(SimEuclid(data, 1, 5))

### 3.6. SISÉ EXERCICI

Desenvolupa un sistema de recomanació col·laboratiu **basat en usuaris**. 

La funció principal, ``getRecommendationsUser``, ha de tenir com a entrada una taula de puntuacions, un ``user_id``, el tipus de mesura de semblança (Euclidiana) que volem usar, el nombre `m` d'usuaris semblants que volem per fer la recomanació i el nombre ``n`` de recomanacions que volem. Com a sortida ha de donar la llista de les ``n`` millors pel·lícules que li podriem recomanar segons la seva semblança amb altres usuaris.

> *Nota 1: S'ha d'evitar comparar ``user_id`` a ell mateix.*

> *Nota 2: Recordeu que en Python podem passar funcions com a paràmetres d'una funció.*

In [ ]:
# getRecommendationsUser(data, 2, 50, 10, SimEuclid)

#### - SECCIÓ 1: 

Computa la score de similitud del usuari desitjat (userID) respecte tots els altres i retorna una llista dels **m** usuaris més propers, que seran els que usarem per fer la recomanació. Normalitzeu els scores de sortida.

In [ ]:
def find_similar_users(DataFrame, userID, m, simfunction):
    """
    Retorna un diccionari de usuaris similars amb les scores corresponents.
    
    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param similarity: mesura de similitud
    :return : diccionary
    """
    
    # la vostra solució aquí
    
    return []

In [ ]:
import datetime
t = datetime.datetime.now()
sim_dict = find_similar_users(data, 2, 10, SimEuclid)
t = datetime.datetime.now()-t
print(str(t))

In [ ]:
sim_dict

**CREIEU QUE ES OPTIM EL TEMPS QUE HA TARDAT PER UN USUARI?** RAONA LA RESPOSTA.

> ANSWER

Per millorar el problema anteorior, construeix una matriu de mida UXU on cada posició *i,j* indica la distància entre l'element *i* i el *j*. Així doncs, si estàs fent un recomanador basat en usuaris, `matriu[2, 3]` contindrà la similitud entre l'usuari 2 i el 3.

In [ ]:
def similarity_matrix(similarity_function, df_counts):
    """
    Retorna una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    
    :param similarity_function: Funció que calcularà la similitud 
        entre usuaris (resp. ítems)
    :param df_counts: Dataframe que conté el nombre de vegades que 
        un usuari ha comprat en un `aisle_id`
    :return : Matriu numpy de mida M x M amb les similituds.
    """
    
    # la vostra solució aquí
    
    return 0

Per cridar aquesta funció, el primer paràmetre pot ser:

* Amb la funció de la distancia euclidiana que heu programat abans pot trigar ~@1h 30min treballant directament amb valors de numpy, ~@20h a partir de pandas pur.
* Opcionalment (no és obligatori fer-ho) podeu programar una funció que treballi específicament amb matrius (i no vectors). Si ho feu, cal gestionar-ho quan es rep `None`.  (@5s)

In [ ]:
sim = similarity_matrix(SimEuclid, df_counts)

In [ ]:
sim.shape

Ara torna a re-fer la funció **find_similar_users** i mira quant triga... 

> Recorda que les scores han d'estar normalitzades!

In [ ]:
def find_similar_users(DataFrame, sim_mx, userID, m):
    
    # la vostra solució aquí
    
    return dict(zip(users, w_scores))

In [ ]:
t = datetime.datetime.now()
sim_dict = find_similar_users(data, sim, 2, 10)
t = datetime.datetime.now()-t
print(str(t))

**Hauria d'haver baixat més de 30 cops el temps anterior!!**

#### - SECCIÓ 2: 
Computa les recomanacions per un usuari amb cadascun dels m usuaris més propers i fes una funció que retorni la **weighted average list** d'aquests per tal d'obtenir la recomanació final. Feu servir la funció anterior que usava la matriu de similituds per anar més ràpid!!


> *Nota: la **weighted average list** es calcularà agregant els n items més puntuats de cadascun dels m users més semblants al usuari donat.

In [ ]:
def getRecommendationsUser(DataFrame, user, sim_mx, n, m):
    """
    Retorna un dataframe de pel·licules amb els scores.
    
    :param DataFrame: dataframe que conté totes les dades
    :param user: usuari al qual fem la recomanació
    :param sim_mx: similarity_function
    :param n: nombre de pelis a recomanar
    :param m: nombre d'usuaris semblants a tenir en compte per les recomanacions
    :return : pandas de pelis amb els seus scores predits
    """
    
    # la vostra solució aquí
    
    return []

In [ ]:

t = datetime.datetime.now()
user_prediction = getRecommendationsUser(data, 3, sim, 10, 50)
t = datetime.datetime.now()-t
print(str(t))


In [ ]:
user_prediction

#### - SECCIÓ 3

Comprovem que realment l'usuari 3 no tenia recomanació per les pelis a les que estem recomanant:

In [ ]:
data_user3 = data[data['user_id']==3]
data_user3.head()

In [ ]:
desired_movie_id_to_check = 0
best_film_to_predict = user_prediction['movie_id'].iloc[desired_movie_id_to_check] 

In [ ]:
data_user3[data_user3['movie_id']==best_film_to_predict]

**Com es que ara estem predint un rating per la pelicula "best_film_to_predict"?** Digues quin nombre és la predicció

In [ ]:
# la vostra solució aquí

### 3.7. SETÉ EXERCICI


A continuació usarem la metrica Mean Absolute Error (MAE) per evaluar el nostre sistema. Aquesta mètrica ens permetrà mesurar la diferencia entre dues llistes donat un usuari: 

**1.** La llista amb els ratings originals d'un usuari donat

**2.** La llista de les prediccions generades per aquest usuari

#### - SECCIÓ 1: 

Treu el 10% dels usuaris i reserva aquests en una variable anomenada **test_set** i la resta en una variable anomenada **train_set**.

In [ ]:
import random
unique_users = data['user_id'].unique()
n_test_users = int(len(unique_users)*0.1)
n_train_users = len(unique_users) - n_test_users

test_usrs = random.sample(list(unique_users), n_test_users)

In [ ]:
n_test_users,n_train_users

In [ ]:
test_set = # la vostra solució aquí
train_set = # la vostra solució aquí
assert len(test_set) + len(train_set) == len(data)

Què passarà si calculo la matriu de similitud amb **train_set** i després intento predir pels usuaris de **test_set**??

#### - SECCIÓ 2:

Selecciona aproximadament el 80% de les interaccions de cada usuari de test i junta-les al **train_set**. Ara podem evaluar el sistema?


> Com la pràctica és molt llarga hem programat el codi per un usuari donat i vosaltres només heu de crear la funció que, per cada usuari, afagi el 80% de les intraccions i les unifiqui al dataframe de train.

In [ ]:
test_set.head()

In [ ]:
# Agafem el 20% de les pelis que ha consumit cada usuari de test 
groupby_count = test_set.groupby('user_id')['movie_id'].count()*0.2
groupby_count

Seleccionem la posició 1 i aquest use_id serà el que usarem pel codi d'exemple (que després haureu de replicar).

In [ ]:
groupby_count.reset_index().iloc[1]

In [ ]:
n_test_samples = int(groupby_count.reset_index().iloc[1]['movie_id'])
u = groupby_count.reset_index().iloc[1]['user_id']

In [ ]:
test_set_user = test_set[test_set['user_id'] == u]
frame_test = test_set_user.sample(n_test_samples)
print("TOTAL SAMPLES OF THE USER: " + str(len(test_set_user)))
print("TOTAL SAMPLES OF THE USER IN TEST SET: " + str(len(frame_test)))

In [ ]:
len(test_set_user.index)

In [ ]:
frame_train = test_set_user[~test_set_user.index.isin(frame_test.index)]
print("TOTAL SAMPLES OF THE USER IN TRAIN SET: " + str(len(frame_train)))

In [ ]:
assert len(frame_train) + len(frame_test) == len(test_set_user)

In [ ]:
def add_testdata(traindf, test_set):
    """
    Retorna els N usuaris més similars basat en la correlació de Pearson
    
    :param traindf: dataframe que conté les dades de train
    :param test_set: dataframe que conté les dades de test

    :return : 
        - :param 1st: dataframe que conté les dades de train juntament amb el 80% de test seleccionat
        - :param 2nd: dataframe que conté les dades de test que queden (20% restant)
    """
    
    # la vostra solució aquí
    
    return pd.concat((traindf, train_appended)), test_appended

In [ ]:
train, test = add_testdata(train_set, test_set)

In [ ]:
train.shape

In [ ]:
test.shape

#### -SECCIÓ 3:

Fes una funció usant **np.abs** que serveixi per evaluar el nostre sistema. **(PISTA:)** La funció ha de mesurar la diferencia en mitjana de la llista de ratings originals amb la llista de ratings predida.

In [ ]:
def evaluateRecommendations(train, test, m, n, sim):
    """
    Retorna l'error generat pel model
    
    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param n: nombre de pelis a retornar
    :param sim: matriu de similitud
    :return : Escalar (float) corresponent al MAE
    """
    
    # la vostra solució 
        
    return 0

In [ ]:
t = datetime.datetime.now()
mae = evaluateRecommendations(train, test, 50, 10, sim)
t = datetime.datetime.now()-t
print(str(t))

In [ ]:
mae

### 3.8. VUITÉ (I ÚTLIM) EXERCICI


**Que surt més a compte, fer un recomanador unic pels dos sexes o un per cada sexe?** Justifica la resposta per escrit i amb el codi necessari.

> RESPOSTA

In [ ]:
# la vostra solució aquí